In [5]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import nltk 
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from collections import Counter 

In [6]:
f=open("yelp_labelled.txt", "r")

In [7]:
file = f.read()

In [8]:
stop_words = set(stopwords.words('english'))

In [9]:
words = word_tokenize(file)

In [10]:
filtered = []
for w in words:
    if w not in stop_words:
        filtered.append(w)
        
        

In [11]:
data = []
lines = file.split('\n')  # split each line
for line in lines:
    line = line.strip()  # remove any outside whitespace
    if line == '':
        continue  # it's a blank line
    label = line[-1]  # the last non-whitespace character
    text = line[:-1].strip()  # everything else, without the extra whitespace
    data.append((text, label))
data[0]

('Wow... Loved this place.', '1')

In [12]:
df = pd.DataFrame(data)


In [13]:
df.columns = ['reviews', 'label']
df.head()

,reviews,label
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [14]:
 df['reviews'] = df['reviews'].str.lower()

In [15]:
keywords = ['best', 'excellent', 'interesting', 'great', 'love', 'like', 'excellent', 'great', 'like','loved',
            'liked','impressed','beautiful','fav','favorite', 'satisfying', 'caring','friendly'
            'pretty','better','fantastic', 'clean','absolutely','authentic','awesome', 'cute','prompt','amazing','highlight'
           'killer', 'pleasure', 'enjoyed','delicious','flavorful', 'plethora','highly',
           'Good', 'Great','must','authentic','delightful','nice','a lot','perfect','healthy','happy','quite','every',
            'experience','fresh','all','friendly','huge','super','special','ambiance','twice','hour','wonderful',
            'reasonable','anytime','family','world','moist','BEST','thumbs','up','pleased','fun',
            'everything','fast','pleasent','surprised','surprisingly','atmosphere','pleasently','stars','5','highly'
           'worth','extremely','promise','price','generous','selection','variety','wow','outstanding','interesting'
           'vibe','options','reviews','huge']

for key in keywords:
    # Note that we add spaces around the key so that we're getting the word,
    # not just pattern matching.
    df[str(key)] = df.reviews.str.contains(
        ' ' + str(key) + ' ',
        case=True
    )

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [16]:
data = df[keywords]
target = df['label']

In [17]:
# Our data is binary / boolean, so we're importing the Bernoulli classifier.
from sklearn.naive_bayes import BernoulliNB

# Instantiate our model and store it in a new variable.
bnb = BernoulliNB()

# Fit our model to the data.
bnb.fit(data, target)

# Classify, storing the result in a new variable.
y_pred = bnb.predict(data)

# Display our results.
print("Number of mislabeled places out of a total {} points : {}".format(
    data.shape[0],
    (target != y_pred).sum()
))

Number of mislabeled places out of a total 1000 points : 307


In [18]:
from sklearn.metrics import confusion_matrix
confusion_matrix(target, y_pred)

array([[469,  31],
       [276, 224]], dtype=int64)

In [19]:

from sklearn.model_selection import train_test_split
# Use train_test_split to create the necessary training and test groups
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=20)
print('With 20% Holdout: ' + str(bnb.fit(X_train, y_train).score(X_test, y_test)))
print('Testing on Sample: ' + str(bnb.fit(data, target).score(data, target)))

With 20% Holdout: 0.665
Testing on Sample: 0.693


In [20]:
from sklearn.model_selection import cross_val_score
np.mean(cross_val_score(bnb, data, target, cv=10))

0.6639999999999999

In [249]:
np.std([0.59, 0.55, 0.54, 0.53, 0.57, 0.52, 0.62, 0.56, 0.51, 0.62])

0.036999999999999984

In [292]:
np.mean([0.62, 0.61, 0.7 , 0.59, 0.7 , 0.59, 0.71, 0.63, 0.6 , 0.71])

0.646

In [23]:
0.0369/.561*100

6.57754010695187